# Taxi Travel Data Analysis

In this demo, we will be doing some demos on temporal feature engineering with the Kaggle Dataset

### Loading libraries, datasets

In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta

In [2]:
# These are all of the files you are given
df_train_og = pd.read_csv("train.csv", index_col=False)

In [3]:
df_train_og.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


### Remove unnecessary columns

### Get Computed Time from POLYLINE

Our goal is to predict the travel-time of the taxi, which can be derived from the POLYLINE length.

Recall:

```
The travel time of the trip (the prediction target of this project) is defined as the (number of points-1) x 15 seconds. 
For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds. Some trips have missing 
data points in POLYLINE, indicated by MISSING_DATA column, and it is part of the challenge how you utilize this knowledge.
```

We are not doing anything with the MISSING_DATA. It is up to you to find a way to use (or ignore) that information.

In [4]:
# remove rows w/ missing data
df_train = df_train_og.copy(deep=True)
df_train = df_train[df_train['MISSING_DATA'] == False]

# separate polyline into target dataset
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 2, 0) * 15

df_targets = df_train["POLYLINE"].apply(polyline_to_trip_duration)

In [5]:
df_train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


### Feature Engineering

In [6]:
# one-hot encode CALL_TYPE
df_train['CALL_TYPE_A'] = df_train['CALL_TYPE'] == 'A'
df_train['CALL_TYPE_B'] = df_train['CALL_TYPE'] == 'B'
df_train['CALL_TYPE_C'] = df_train['CALL_TYPE'] == 'C'

df_train['CALL_TYPE_A'] = df_train['CALL_TYPE_A'].map({True:1, False:0})
df_train['CALL_TYPE_B'] = df_train['CALL_TYPE_B'].map({True:1, False:0})
df_train['CALL_TYPE_C'] = df_train['CALL_TYPE_C'].map({True:1, False:0})


# set ORIGIN_CALL values and compress unique values
'''df_train['ORIGIN_CALL_COMPRESSED'] = df_train['ORIGIN_CALL'].fillna(0)

phones = np.sort(df_train['ORIGIN_CALL_COMPRESSED'].unique())
phone_dict = {}
for i,v in enumerate(phones):
    phone_dict[v] = i
df_train['ORIGIN_CALL_COMPRESSED'] = df_train['ORIGIN_CALL_COMPRESSED'].map(phone_dict)'''
df_train['ORIGIN_CALL_AVGLEN'] = df_train['ORIGIN_CALL'].fillna(0)

call_count = df_train['ORIGIN_CALL_AVGLEN'].value_counts()
average_typeA_len = np.mean(df_targets[df_train['CALL_TYPE_A'] == True])
phones = np.sort(df_train['ORIGIN_CALL_AVGLEN'].unique())
phone_dict = {}
for phone in phones:
    phone_dict[phone] = average_typeA_len if call_count[phone] < 30 else np.mean(df_targets[df_train['ORIGIN_CALL_AVGLEN'] == phone])
phone_dict[0] = 0
df_train['ORIGIN_CALL_AVGLEN'] = df_train['ORIGIN_CALL_AVGLEN'].map(phone_dict)


# translate ORIGIN_STAND to lat/long
df_train['ORIGIN_STAND'] = df_train['ORIGIN_STAND'].fillna(0)
df_stand_coords = pd.read_csv("metaData_taxistandsID_name_GPSlocation.csv", index_col=False)
df_stand_coords['Latitude'] = pd.to_numeric(df_stand_coords['Latitude'])
df_stand_coords['Longitude'] = pd.to_numeric(df_stand_coords['Longitude'])

# normalize lat/long coordinates
lat_mean = df_stand_coords['Latitude'].mean()
lat_stddev = df_stand_coords['Latitude'].std()
long_mean = df_stand_coords['Longitude'].mean()
long_stddev = df_stand_coords['Longitude'].std()

# two dictionaries for lat/long because using map is WAY faster than using apply and 0 -> 0 is easier too
lat_dict = {0:0}
long_dict = {0:0}
for stand_id in df_stand_coords['ID'].unique():
    lat_dict[stand_id] = (df_stand_coords[df_stand_coords['ID']==stand_id].iloc[0]['Latitude'] - lat_mean) / lat_stddev
    long_dict[stand_id] = (df_stand_coords[df_stand_coords['ID']==stand_id].iloc[0]['Longitude'] - long_mean) / long_stddev
df_train['ORIGIN_STAND_LAT'] = df_train['ORIGIN_STAND'].map(lat_dict)
df_train['ORIGIN_STAND_LONG'] = df_train['ORIGIN_STAND'].map(long_dict)


# set TAXI_ID values and compress unique values
'''df_train['TAXI_ID_COMPRESSED'] = df_train['TAXI_ID'].fillna(0)

taxi_ids = np.sort(df_train['TAXI_ID_COMPRESSED'].unique())
taxi_id_dict = {}
for i,v in enumerate(taxi_ids):
    taxi_id_dict[v] = i
df_train['TAXI_ID_COMPRESSED'] = df_train['TAXI_ID_COMPRESSED'].map(taxi_id_dict)'''
df_train['TAXI_ID_AVGLEN'] = df_train['TAXI_ID'].fillna(0)

taxi_ids = np.sort(df_train['TAXI_ID_AVGLEN'].unique())
taxi_len_dict = {}
for taxi_id in taxi_ids:
    taxi_len_dict[taxi_id] = np.mean(df_targets[df_train['TAXI_ID'] == taxi_id])
df_train['TAXI_ID_AVGLEN'] = df_train['TAXI_ID_AVGLEN'].map(taxi_len_dict)


# timestamp binning (minutes + day of week) - monday = 0, sunday = 6
df_train['DATETIME_TEMP'] = pd.to_datetime(df_train['TIMESTAMP'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-1')
df_train['TIMESTAMP_WEEKDAY'] = df_train['DATETIME_TEMP'].dt.weekday
df_train['TIMESTAMP_MINUTE'] = df_train['DATETIME_TEMP'].dt.hour * 60 + df_train['DATETIME_TEMP'].dt.minute

In [7]:
# origin_call,taxi_id vs trip length testing
# TODO: There are 57106 unique origin_call values, which would be hard to train
# perhaps we could order them by average trip length to have some correlation in the data to make training easier

# maybe do the same w/ taxi id (448 unique IDs)

In [8]:
df_train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ORIGIN_CALL_AVGLEN,ORIGIN_STAND_LAT,ORIGIN_STAND_LONG,TAXI_ID_AVGLEN,DATETIME_TEMP,TIMESTAMP_WEEKDAY,TIMESTAMP_MINUTE
0,1372636858620000589,C,NaN,0.0,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",0,0,1,0.0,0.000000,0.00000,691.691610,2013-07-01 01:00:58+01:00,0,60
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",0,1,0,0.0,-0.034448,-0.74524,724.374057,2013-07-01 01:08:23+01:00,0,68
2,1372636951620000320,C,NaN,0.0,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",0,0,1,0.0,0.000000,0.00000,698.520176,2013-07-01 01:02:31+01:00,0,62
3,1372636854620000520,C,NaN,0.0,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",0,0,1,0.0,0.000000,0.00000,798.858871,2013-07-01 01:00:54+01:00,0,60
4,1372637091620000337,C,NaN,0.0,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",0,0,1,0.0,0.000000,0.00000,738.533415,2013-07-01 01:04:51+01:00,0,64


In [9]:
# remove useless columns
df_train = df_train.drop(['TRIP_ID','CALL_TYPE','ORIGIN_CALL','ORIGIN_STAND','TAXI_ID','TIMESTAMP','DAY_TYPE',
                          'MISSING_DATA','POLYLINE','DATETIME_TEMP'], axis=1)

In [10]:
df_train[100000:100010]

,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ORIGIN_CALL_AVGLEN,ORIGIN_STAND_LAT,ORIGIN_STAND_LONG,TAXI_ID_AVGLEN,TIMESTAMP_WEEKDAY,TIMESTAMP_MINUTE
100000,0,1,0,0.000000,-1.689330,0.316291,638.352363,6,1166
100001,0,1,0,0.000000,0.000000,0.000000,672.618706,6,1229
100002,1,0,0,750.338968,0.000000,0.000000,597.437211,6,1198
100003,0,0,1,0.000000,0.000000,0.000000,143.211893,6,1177
100004,0,0,1,0.000000,0.000000,0.000000,674.074642,6,1224
100005,0,1,0,0.000000,1.801347,1.501453,707.959805,6,1178
100006,0,1,0,0.000000,-1.420951,0.630999,760.682314,6,1184
100007,0,0,1,0.000000,0.000000,0.000000,715.824968,6,1173
100008,0,1,0,0.000000,-1.035266,1.384046,774.118736,6,1144
100009,0,1,0,0.000000,-0.503713,-1.014294,715.639961,6,1182


In [11]:
# Export to file
df_train.to_csv("train_engineered.csv", index=False)
df_targets.to_csv("targets_engineered.csv", index=False)

### Do the same engineering to the test set

In [12]:
# These are all of the files you are given
df_test_og = pd.read_csv("test_public.csv", index_col=False)
df_test_og.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,T1,B,NaN,15.0,20000542,1408039037,A,False
1,T2,B,NaN,57.0,20000108,1408038611,A,False
2,T3,B,NaN,15.0,20000370,1408038568,A,False
3,T4,B,NaN,53.0,20000492,1408039090,A,False
4,T5,B,NaN,18.0,20000621,1408039177,A,False


In [13]:
# remove rows w/ missing data
df_test = df_test_og.copy(deep=True)
df_test = df_test[df_test['MISSING_DATA'] == False]

In [14]:
# one-hot encode CALL_TYPE
df_test['CALL_TYPE_A'] = df_test['CALL_TYPE'] == 'A'
df_test['CALL_TYPE_B'] = df_test['CALL_TYPE'] == 'B'
df_test['CALL_TYPE_C'] = df_test['CALL_TYPE'] == 'C'

df_test['CALL_TYPE_A'] = df_test['CALL_TYPE_A'].map({True:1, False:0})
df_test['CALL_TYPE_B'] = df_test['CALL_TYPE_B'].map({True:1, False:0})
df_test['CALL_TYPE_C'] = df_test['CALL_TYPE_C'].map({True:1, False:0})


# set ORIGIN_CALL values and compress unique values
df_test['ORIGIN_CALL_AVGLEN'] = df_test['ORIGIN_CALL'].fillna(0)
df_test['ORIGIN_CALL_AVGLEN'] = df_test['ORIGIN_CALL_AVGLEN'].map(phone_dict)
df_test['ORIGIN_CALL_AVGLEN'] = df_test['ORIGIN_CALL_AVGLEN'].fillna(average_typeA_len)


# translate ORIGIN_STAND to lat/long
df_test['ORIGIN_STAND'] = df_test['ORIGIN_STAND'].fillna(0)
df_test['ORIGIN_STAND_LAT'] = df_test['ORIGIN_STAND'].map(lat_dict)
df_test['ORIGIN_STAND_LONG'] = df_test['ORIGIN_STAND'].map(long_dict)


# set TAXI_ID values and compress unique values
df_test['TAXI_ID_AVGLEN'] = df_test['TAXI_ID'].fillna(0)
df_test['TAXI_ID_AVGLEN'] = df_test['TAXI_ID_AVGLEN'].map(taxi_len_dict)


# timestamp binning (minutes + day of week) - monday = 0, sunday = 6
df_test['DATETIME_TEMP'] = pd.to_datetime(df_test['TIMESTAMP'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-1')
df_test['TIMESTAMP_WEEKDAY'] = df_test['DATETIME_TEMP'].dt.weekday
df_test['TIMESTAMP_MINUTE'] = df_test['DATETIME_TEMP'].dt.hour * 60 + df_test['DATETIME_TEMP'].dt.minute

In [15]:
# remove useless columns
df_test = df_test.drop(['TRIP_ID','CALL_TYPE','ORIGIN_CALL','ORIGIN_STAND','TAXI_ID','TIMESTAMP','DAY_TYPE',
                          'MISSING_DATA','DATETIME_TEMP'], axis=1)

In [16]:
df_test[21:50]

,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,ORIGIN_CALL_AVGLEN,ORIGIN_STAND_LAT,ORIGIN_STAND_LONG,TAXI_ID_AVGLEN,TIMESTAMP_WEEKDAY,TIMESTAMP_MINUTE
21,1,0,0,750.338968,0.000000,0.000000,738.713063,3,1136
22,1,0,0,508.367876,0.000000,0.000000,699.471438,3,1135
23,0,1,0,0.000000,-1.035266,1.384046,809.000000,3,1139
24,0,1,0,0.000000,-0.462931,-0.308142,727.574565,3,1137
25,0,1,0,0.000000,1.690864,0.751705,761.194030,3,1137
26,0,1,0,0.000000,-1.035266,1.384046,672.285714,3,1134
27,0,1,0,0.000000,-1.035266,1.384046,798.908648,3,1139
28,0,1,0,0.000000,0.045437,-0.273941,760.682314,3,1127
29,0,1,0,0.000000,-1.035266,1.384046,737.178749,3,1130
30,0,1,0,0.000000,-1.035266,1.384046,656.337662,3,1128


In [17]:
# Export to file
df_test.to_csv("test_engineered.csv", index=False)

### Create a Prediction File

In [18]:
mean, std = df_tr["LEN"].mean(), df_tr["LEN"].std()
median = df_tr["LEN"].median()
print(f"{mean=} {median=} {std=}")

NameError: name 'df_tr' is not defined

In [ ]:
# Sample submission file that is given on kaggle
df_sample = pd.read_csv("sampleSubmission.csv")

df_sample["TRAVEL_TIME"] = 716.43

# mean(716.43) -> 792.73593
# median(600) -> 784.74219
df_sample.to_csv("my_pred.csv", index=None)

### Do some Feature Analysis

For our feature analysis, we are looking at which of our engineered features may be useful in making a taxicab time regression model

In [ ]:
# First n samples to analyze. Set to -1 to use all data
end = -1

outlier_threshold = 3

# "Choose all data, where the trip length is less than 3 standard deviations away from the mean"
# This is to remove outliers. Otherwise, our plots would look very squished (since there are some
# VERRRRRY long taxi trips in the dataset)
df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]

# Because our y-values only take on multiples of 15, we want just enough buckets in a histogram
# such that each buckets counts one value's frequency. (e.x. one bucket counts how many 15s trips, 
# how many 30s trips, etc. )
buckets = (int(mean + outlier_threshold * std) // 15)

print(f"Using: {len(df_trimmed)}/{len(df_tr)}")

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(18,14))

# Now, we visualize some features that we think might be useful
for idx, v in enumerate(["YR", "MON", "DAY", "HR", "WK", "ORIGIN_STAND"]):
  # idx // 3 = row, idx % 3 = column
  ax = axs[idx // 3, idx % 3]
  
  # Remove any rows with invalid values
  df_subset = df_trimmed.dropna(subset=v)
  
  # Create a histogram. Look up the documentation for more details
  ax.hist2d(df_subset[v][:end], df_subset["LEN"][:end], cmap="CMRmap", bins=(120,buckets))
  
  # Some stylistic things to make the graphs look nice
  ax.set_xlim(ax.get_xlim()[0] - 1, ax.get_xlim()[1] + 1)
  ax.set_facecolor("black")
  ax.set_ylabel("seconds", fontsize=18)
  ax.set_title(f"Feature: {v}", fontsize=20)


In [ ]:
plt.figure(figsize=(10,10))
for v in [0, 5, 11, 17, 23]:
  # Filter data where the HR matches v
  hourly_data = df_trimmed[df_trimmed["HR"] == v]["LEN"]
  histogram, bin_boundary = np.histogram(hourly_data, bins=buckets)
  histogram = histogram / len(hourly_data)
  # The center is the left_bound and right_bound of a bucket
  bin_centers = [(bin_boundary[i] + bin_boundary[i + 1]) / 2 for i in range(buckets)]
  plt.plot(bin_centers, histogram, label=f"HR={v}")
plt.legend();